- __Cross validation__ bertujuan:
    - Memvalidasi model
    - Mencari nilai parameter terbaik
    
    
- Step-by-step cross validation:
    1. Splitting dataset: training & testing
    2. Data training di-split/fold: training & validasi
    3. Misal data training displit menjadi 3 bagian: A, B, C
        - Buat model dg data train: AB lalu divalidasi C, kmd cek evaluasi metrik misal akurasinya
        - Buat model dg data train: AC lalu divalidasi B, kmd cek evaluasi metrik misal akurasinya
        - Buat model dg data train: BC lalu divalidasi A, kmd cek evaluasi metrik misal akurasinya
    4. Didapatkan 3 model dengan 3 nilai akurasi, hitung rata" akurasinya
    
    
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width="350px" height="350px">

In [4]:
import numpy as np
import pandas as pd

<hr>

### K-Fold CV (Cross Validation)

In [2]:
from sklearn.model_selection import KFold

In [9]:
x = np.arange(1, 10)
x

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

<hr>

__K-Fold CV__ dengan $k = 3$, artinya: 
- Data $x$ akan di-split/fold menjadi 3 lipatan/fold
- 2 bagian secara bergantian akan digunakan sebagai data training dan 
- 1 bagian secara bergantian akan digunakan sebagai data validasi

Hasil K-Fold CV $k = 3$ pada $x$:
- split 1: training = [4, 5, 6, 7, 8, 9]  validasi: [1, 2, 3]
- split 2: training = [1, 2, 3, 7, 8, 9]  validasi: [4, 5, 6]
- split 3: training = [1, 2, 3, 4, 5, 6]  validasi: [7, 8, 9]

In [10]:
k = KFold(n_splits = 3)

for train, validate in k.split(x):
    print(train, validate)
    
# hasil yang diprint adalah INDEX data!

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [14]:
# print VALUE data hasil K-fold

for train, validate in k.split(x):
    # print(train, validate)
    print([x[i] for i in train], [x[i] for i in validate])

[4, 5, 6, 7, 8, 9] [1, 2, 3]
[1, 2, 3, 7, 8, 9] [4, 5, 6]
[1, 2, 3, 4, 5, 6] [7, 8, 9]


In [18]:
#  k-maximum = sejumlah value sample

k = KFold(n_splits = len(x))

for train, validate in k.split(x):
    print(train, validate)

[1 2 3 4 5 6 7 8] [0]
[0 2 3 4 5 6 7 8] [1]
[0 1 3 4 5 6 7 8] [2]
[0 1 2 4 5 6 7 8] [3]
[0 1 2 3 5 6 7 8] [4]
[0 1 2 3 4 6 7 8] [5]
[0 1 2 3 4 5 7 8] [6]
[0 1 2 3 4 5 6 8] [7]
[0 1 2 3 4 5 6 7] [8]


<hr>

### Contoh Case K-Fold CV

Pada kasus klasifikasi dengan algoritma Logistic Regression dan dataset Iris.

__1. Prepare datasets & split into train & test ```train_test_split()```__

In [21]:
from sklearn.datasets import load_iris
data = load_iris()

In [22]:
dir(data)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [25]:
df = pd.DataFrame(
    data['data'],
    # columns = data['feature_names']
    columns = ['SL', 'SW', 'PL', 'PW']
)
df['target'] = data['target']
df.head()

,SL,SW,PL,PW,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [26]:
# splitting
from sklearn.model_selection import train_test_split

In [30]:
xtr, xts, ytr, yts = train_test_split(
    df[['SL', 'SW', 'PL', 'PW']],
    df['target'],
    test_size = .2
)
print(len(xtr), len(xts))

120 30


<hr>

__2. K-Fold CV dengan $k = 6$ diterapkan ke Logitic Regression__

In [32]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [39]:
# gabungkan xtr+ytr dan xts+yts

dtrain = pd.concat([xtr, ytr], axis=1)
dtes = pd.concat([xts, yts], axis=1)
print(len(dtrain), len(dtes))

120 30


In [47]:
# data training di-folding dengan k=6, diterapkan ke LogisticRegression

score = []

k = KFold(n_splits = 6)
for train, validate in k.split(dtrain):
    # print(train, validate)
    xtrain = dtrain.iloc[train][['SL', 'SW', 'PL', 'PW']]
    ytrain = dtrain.iloc[train]['target']
    xvalid = dtrain.iloc[validate][['SL', 'SW', 'PL', 'PW']]
    yvalid = dtrain.iloc[validate]['target']
    # print(xtrain)
    
    # train model dg xtrain, ytrain
    model = LogisticRegression()
    model.fit(xtrain, ytrain)
    
    # check score model dg xvalid, yvalid
    score.append(model.score(xvalid, yvalid))
    
print(score)
print(np.mean(score))

C:\Users\HP\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\HP\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

[1.0, 1.0, 0.95, 0.85, 1.0, 1.0]
0.9666666666666668


<hr>

__3. Sklearn ```cross_val_score()```__

Cara di atas (CV untuk memvalidasi model/melihat akurasi) dapat lebih mudah dilakukan.

In [49]:
from sklearn.model_selection import cross_val_score

In [51]:
cv_score = cross_val_score(
    LogisticRegression(),
    xtr,
    ytr,
    cv = 6
)

print(cv_score)
print(np.mean(cv_score))

[1.  1.  1.  0.8 1.  1. ]
0.9666666666666667


C:\Users\HP\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
